### Installing necessary libraries

In [60]:
%pip install langgraph google-cloud-aiplatform pinecone pydantic --quiet

I0000 00:00:1760037263.803692    5763 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [61]:
!python -m pip install --upgrade google-genai --quiet

I0000 00:00:1760037267.875986    5763 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [62]:
!python -m pip install dotenv --quiet

I0000 00:00:1760037271.247203    5763 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [64]:
# gcloud auth application-default login

### Loading env variables

In [65]:
from dotenv import load_dotenv
load_dotenv()

True

In [66]:
from google import genai

### setting up vertex ai 

In [67]:
import os

PROJECT_ID = "bdc-trainings"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "global")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

2025-10-10 00:44:57,811 - google_genai._api_client - INFO - The user provided project/location will take precedence over the Vertex AI API key from the environment variable.


In [68]:
if not client.vertexai:
    print("Using Gemini Developer API.")
elif client._api_client.project:
    print(
        f"Using Vertex AI with project: {client._api_client.project} in location: {client._api_client.location}"
    )
elif client._api_client.api_key:
    print(
        f"Using Vertex AI in express mode with API key: {client._api_client.api_key[:5]}...{client._api_client.api_key[-5:]}"
    )

Using Vertex AI with project: bdc-trainings in location: global


### configurations

In [69]:

import os

EMBEDDING_MODEL = "models/gemini-embedding-001"
LLM_MODEL = "gemini-2.5-flash"

In [70]:
import os
import time
from dataclasses import dataclass
from typing import List, Dict, Any

# Pinecone (serverless)
from pinecone import Pinecone, ServerlessSpec

### Initialising Pinecone

In [71]:
PINECONE_API_KEY = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

if not PINECONE_API_KEY:
    raise RuntimeError("PINECONE_API_KEY not found. Please set it in your environment.")
else:
  print("PINECONE API KEY found")

pc = Pinecone(api_key=PINECONE_API_KEY)

PINECONE API KEY found


### Initialising Logger

In [72]:
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,  # Change to DEBUG for more detailed logs
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

logger = logging.getLogger(__name__)


In [73]:
import logging

# Configure logging to both console and file
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("rag_workflow.log"),  # Logs to file
        logging.StreamHandler()                   # Logs to console
    ]
)

logger = logging.getLogger(__name__)

In [74]:
import json
import pinecone

from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model=EMBEDDING_MODEL)

test_dim = len(embeddings.embed_query("dimension probe"))

In [75]:
print(test_dim)

3072


### creating vector db and index through Pinecone

In [76]:
INDEX_NAME = "raggcp1" 
METRIC = "cosine"

# Create the index if it doesn't exist
existing = [idx["name"] for idx in pc.list_indexes()]
if INDEX_NAME not in existing:
    print(f"Creating index '{INDEX_NAME}' ...")
    pc.create_index(
        name=INDEX_NAME,
        dimension=test_dim, #3072
        metric=METRIC,
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),)
    # optional: wait a moment for the index to be ready
    time.sleep(5)
else:
    print(f"Index '{INDEX_NAME}' already exists, reusing it.")

index = pc.Index(INDEX_NAME)
print(index.describe_index_stats())

Index 'raggcp1' already exists, reusing it.
{'dimension': 3072,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 30}},
 'total_vector_count': 30,
 'vector_type': 'dense'}


### Load your dataset

In [77]:
# Load your dataset
with open("datasets/self_critique_loop_dataset.json") as f:
    kb_data = json.load(f)

In [78]:
print(kb_data)

[{'doc_id': 'KB001', 'question': 'What are best practices for debugging?', 'answer_snippet': "When addressing debugging, it's important to follow well-defined patterns...", 'source': 'debugging_guide.md', 'confidence_indicator': 'moderate', 'last_updated': '2024-01-10'}, {'doc_id': 'KB002', 'question': 'What are best practices for performance tuning?', 'answer_snippet': "When addressing performance tuning, it's important to follow well-defined patterns...", 'source': 'performance tuning_guide.md', 'confidence_indicator': 'moderate', 'last_updated': '2024-02-10'}, {'doc_id': 'KB003', 'question': 'What are best practices for caching?', 'answer_snippet': "When addressing caching, it's important to follow well-defined patterns...", 'source': 'caching_guide.md', 'confidence_indicator': 'moderate', 'last_updated': '2024-03-10'}, {'doc_id': 'KB004', 'question': 'What are best practices for asynchronous programming?', 'answer_snippet': "When addressing asynchronous programming, it's important 

### Embedding the data

In [79]:
# Extract texts
texts = [entry["answer_snippet"] for entry in kb_data]

# Get embeddings
embeddings_1 = embeddings.embed_documents(texts)

#### Upserting data into vector db

In [80]:
# Upsert
vectors = [{
    "id": item["doc_id"],
    "values": vector,
    "metadata": {
        "question": item["question"],
        "snippet": item["answer_snippet"],
        "source": item["source"]
    }
} for item, vector in zip(kb_data, embeddings_1)]

index.upsert(vectors)

print("KB Indexed into Pinecone")


KB Indexed into Pinecone


In [81]:
# ###testing
# query = "What are best practices for performance tuning?"
# k = 5

# # Get the embedding vector for the query
# query_vector = embeddings.embed_query(query)

# # Query Pinecone index using the vector
# results = index.query(
#     vector=query_vector,
#     top_k=k,
#     include_metadata=True
# )


In [82]:
# results

### Defining RAG NODES

In [83]:
### Defining RAG NODES

# Retrieval Node
def retrieve_kb(question, top_k=5):
    query_vector = embeddings.embed_query(question)
    results = index.query(
    vector=query_vector,
    top_k=5,
    include_metadata=True
)
    return results["matches"]

In [84]:
###testing the retrieval node
# query="What are best practices for asynchronous programming?"
# retrieve_kb(query="What are best practices for asynchronous programming?")

In [85]:
# snippets=retrieve_kb(query="What are best practices for asynchronous programming?")

### Generation Node

In [86]:
# Generation Node
def generate_answer(question, snippets):
    context = "\n".join([f"[{s['id']}] {s['metadata']['snippet']}" for s in snippets])
    prompt = f"""Use the context below to answer the question:
Context:
{context}
Question: {question}
Answer with citations in the format [KBxxx]."""

    return client.models.generate_content(contents=prompt, model=LLM_MODEL).text.strip()

In [87]:
# generate_answer(query="What are best practices for asynchronous programming?",snippets=snippets)

In [88]:
# answer=generate_answer(query="What are best practices for asynchronous programming?",snippets=snippets)

### Critique Node

In [89]:
# Critique Node
def critique_answer(question, snippets, answer):
    context = "\n".join([f"- {s['metadata']['snippet']}" for s in snippets])
    prompt = f"""Given the context and an answer, determine if the answer is COMPLETE or needs REFINEMENT.
Context:
{context}

Question: {question}

Answer:
{answer}

Respond in one of the following formats:
- COMPLETE
- REFINE: <list missing keywords or ideas>
"""
    return client.models.generate_content(contents=prompt, model=LLM_MODEL).text.strip()

In [90]:
# critique_answer(query, snippets, answer)

### Refinement Node

In [91]:
# Refinement Node
def refine_answer(question, prev_snippets, missing_keywords):
    new_query = f"{question} {missing_keywords}"
    extra_snippet = retrieve_kb(new_query, top_k=1)
    combined = prev_snippets + extra_snippet
    return generate_answer(question, combined)


In [92]:
# refine_answer(query, prev_snippets=snippets, missing_keywords='REFINE: performance tuning')

### Build LangGraph Workflow 

In [93]:
# Build LangGraph Workflow 


from typing import TypedDict
from langgraph.graph import StateGraph


# Define the schema for the graph state
class RAGState(TypedDict):
    question: str
    snippets: list
    answer: str
    critique: str
    final_answer: str


def build_rag_graph():
    graph = StateGraph(RAGState)
    
    def retrieve(state):
        question = state["question"]
        logger.info(f"Retrieving snippets for question: {question}")
        snippets = retrieve_kb(question)
        logger.info(f"[Retriever] Retrieved {len(snippets)} snippets")

        return {"question": question, "snippets": snippets}
    
    
    def generate(state):
        logger.info(f"[GenerateAnswer] Generating answer for: {state['question']}")
        answer = generate_answer(state["question"], state["snippets"])
        logger.info(f"[GenerateAnswer] Generated answer: {answer[:100]}...")
        return {**state, "answer": answer}

    
    def critique(state):
        logger.info(f"[Critique] Critiquing answer...")
        result = critique_answer(state["question"], state["snippets"], state["answer"])
        logger.info(f"[Critique] Critique result: {result}")
        return {**state, "critique": result}

    
    # def needs_refine(state):
    #     return "REFINE" in state["critique"]
    
    def needs_refine(state):
        decision = "REFINE" in state["critique"]
        logger.info(f"[CheckCritique] Needs refinement? {decision}")
        return decision

    
    def refine(state):
        logger.info(f"[Refine] Refining answer...")
        missing = state["critique"].split("REFINE:")[-1].strip()
        final_answer = refine_answer(state["question"], state["snippets"], missing)
        logger.info(f"[Refine] Final refined answer: {final_answer[:100]}...")
        return {**state, "final_answer": final_answer}
    
    def end_node(state):
        logger.info(f"[END] Final state reached.")
        return state  # Just passes through
 
    graph.add_node("Retriever", retrieve)
    graph.add_node("GenerateAnswer", generate)
    graph.add_node("Critique", critique)
    graph.add_node("Refine", refine)
    graph.add_node("END", end_node)

    # Add condition
    graph.add_conditional_edges("Critique", needs_refine, {
        True: "Refine",
        False: "END"
    })
   
# Set entry and edges
    graph.set_entry_point("Retriever")
    graph.add_edge("Retriever", "GenerateAnswer")
    graph.add_edge("GenerateAnswer", "Critique")
    graph.add_edge("Refine", "END")

    return graph.compile()

In [94]:
graph= build_rag_graph()

question = input("Enter your question: ")
logger.info(f"[Main] Starting RAG workflow for question: {question}")

result = graph.invoke({"question": question})

logger.info(f"[Main] Workflow completed. Final result: {result.get('final_answer', result.get('answer'))}")
print(result.get("final_answer", result.get("answer")))

2025-10-10 00:46:53,696 - __main__ - INFO - [Main] Starting RAG workflow for question: "What are best practices for asynchronous programming?"
2025-10-10 00:46:53,698 - __main__ - INFO - Retrieving snippets for question: "What are best practices for asynchronous programming?"
2025-10-10 00:46:55,371 - __main__ - INFO - [Retriever] Retrieved 5 snippets
2025-10-10 00:46:55,373 - __main__ - INFO - [GenerateAnswer] Generating answer for: "What are best practices for asynchronous programming?"
2025-10-10 00:46:55,374 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-10-10 00:46:59,146 - httpx - INFO - HTTP Request: POST https://aiplatform.googleapis.com/v1beta1/projects/bdc-trainings/locations/global/publishers/google/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
2025-10-10 00:46:59,152 - __main__ - INFO - [GenerateAnswer] Generated answer: When addressing asynchronous programming, it's important to follow well-defined patterns [KB004, KB0...
2025-10-

When addressing asynchronous programming, it's important to follow well-defined patterns [KB004, KB014, KB024].


In [95]:
print(result.get("final_answer", result.get("answer")))

When addressing asynchronous programming, it's important to follow well-defined patterns [KB004, KB014, KB024].


In [96]:
graph= build_rag_graph()

question=input("Enter your question: ")

result= graph.invoke({"question": question})


print(result.get("final_answer", result["answer"]))

2025-10-10 00:47:30,496 - __main__ - INFO - Retrieving snippets for question: "What are best practices for asynchronous programming?"
2025-10-10 00:47:31,520 - __main__ - INFO - [Retriever] Retrieved 5 snippets
2025-10-10 00:47:31,522 - __main__ - INFO - [GenerateAnswer] Generating answer for: "What are best practices for asynchronous programming?"
2025-10-10 00:47:31,522 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-10-10 00:47:33,738 - httpx - INFO - HTTP Request: POST https://aiplatform.googleapis.com/v1beta1/projects/bdc-trainings/locations/global/publishers/google/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
2025-10-10 00:47:33,740 - __main__ - INFO - [GenerateAnswer] Generated answer: When addressing asynchronous programming, it's important to follow well-defined patterns [KB014, KB0...
2025-10-10 00:47:33,740 - __main__ - INFO - [Critique] Critiquing answer...
2025-10-10 00:47:33,741 - google_genai.models - INFO - AFC is enabled with 

When addressing asynchronous programming, it's important to follow well-defined patterns [KB014, KB024, KB004].


### Testing Queries
Try the pipeline with:
1. “What are best practices for caching?”
2. “How should I set up CI/CD pipelines?”
3. “What are performance tuning tips?”
4. “How do I version my APIs?”
5. “What should I consider for error handling?

#### What are best practices for caching?

In [97]:
graph= build_rag_graph()

question=input("Enter your question: ")

result= graph.invoke({"question": question})


print(result.get("final_answer", result["answer"]))

2025-10-10 00:48:03,314 - __main__ - INFO - Retrieving snippets for question: “What are best practices for caching?”
2025-10-10 00:48:04,314 - __main__ - INFO - [Retriever] Retrieved 5 snippets
2025-10-10 00:48:04,316 - __main__ - INFO - [GenerateAnswer] Generating answer for: “What are best practices for caching?”
2025-10-10 00:48:04,316 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-10-10 00:48:06,343 - httpx - INFO - HTTP Request: POST https://aiplatform.googleapis.com/v1beta1/projects/bdc-trainings/locations/global/publishers/google/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
2025-10-10 00:48:06,345 - __main__ - INFO - [GenerateAnswer] Generated answer: When addressing caching, it's important to follow well-defined patterns [KB023, KB013, KB003]....
2025-10-10 00:48:06,346 - __main__ - INFO - [Critique] Critiquing answer...
2025-10-10 00:48:06,346 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-10-10 00:48:0

When addressing caching, it's important to follow well-defined patterns [KB023, KB013, KB003].


####  “How should I set up CI/CD pipelines?”

In [ ]:
graph= build_rag_graph()

question=input("Enter your question: ")

result= graph.invoke({"question": question})

print(result.get("final_answer", result["answer"]))

2025-10-10 00:48:22,080 - __main__ - INFO - Retrieving snippets for question: “How should I set up CI/CD pipelines?”
2025-10-10 00:48:23,045 - __main__ - INFO - [Retriever] Retrieved 5 snippets
2025-10-10 00:48:23,046 - __main__ - INFO - [GenerateAnswer] Generating answer for: “How should I set up CI/CD pipelines?”
2025-10-10 00:48:23,047 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-10-10 00:48:24,841 - httpx - INFO - HTTP Request: POST https://aiplatform.googleapis.com/v1beta1/projects/bdc-trainings/locations/global/publishers/google/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
2025-10-10 00:48:24,843 - __main__ - INFO - [GenerateAnswer] Generated answer: When setting up CI/CD pipelines, it's important to follow well-defined patterns [KB007, KB017, KB027...
2025-10-10 00:48:24,844 - __main__ - INFO - [Critique] Critiquing answer...
2025-10-10 00:48:24,845 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-10-10 0

When addressing CI/CD, it's important to follow well-defined patterns [KB027, KB017, KB007].


#### 3. “What are performance tuning tips?”

In [99]:
graph= build_rag_graph()

question=input("Enter your question: ")

result= graph.invoke({"question": question})


print(result.get("final_answer", result["answer"]))

2025-10-10 00:48:55,452 - __main__ - INFO - Retrieving snippets for question: “What are performance tuning tips?” 
2025-10-10 00:48:56,424 - __main__ - INFO - [Retriever] Retrieved 5 snippets
2025-10-10 00:48:56,425 - __main__ - INFO - [GenerateAnswer] Generating answer for: “What are performance tuning tips?” 
2025-10-10 00:48:56,426 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-10-10 00:48:58,692 - httpx - INFO - HTTP Request: POST https://aiplatform.googleapis.com/v1beta1/projects/bdc-trainings/locations/global/publishers/google/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
2025-10-10 00:48:58,695 - __main__ - INFO - [GenerateAnswer] Generated answer: When addressing performance tuning, it's important to follow well-defined patterns [KB012, KB002, KB...
2025-10-10 00:48:58,696 - __main__ - INFO - [Critique] Critiquing answer...
2025-10-10 00:48:58,696 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-10-10 00:49

When addressing performance tuning, it's important to follow well-defined patterns [KB012, KB002, KB022].


#### 4. “How do I version my APIs?”

In [100]:
graph= build_rag_graph()

question=input("Enter your question: ")

result= graph.invoke({"question": question})

print(result.get("final_answer", result["answer"]))

2025-10-10 00:49:42,632 - __main__ - INFO - Retrieving snippets for question: “How do I version my APIs?”
2025-10-10 00:49:43,636 - __main__ - INFO - [Retriever] Retrieved 5 snippets
2025-10-10 00:49:43,637 - __main__ - INFO - [GenerateAnswer] Generating answer for: “How do I version my APIs?”
2025-10-10 00:49:43,638 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-10-10 00:49:45,789 - httpx - INFO - HTTP Request: POST https://aiplatform.googleapis.com/v1beta1/projects/bdc-trainings/locations/global/publishers/google/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
2025-10-10 00:49:45,791 - __main__ - INFO - [GenerateAnswer] Generated answer: When addressing API versioning, it's important to follow well-defined patterns [KB005, KB025, KB015]...
2025-10-10 00:49:45,792 - __main__ - INFO - [Critique] Critiquing answer...
2025-10-10 00:49:45,792 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-10-10 00:49:53,590 - httpx - 

When addressing API versioning, it's important to follow well-defined patterns [KB005, KB025, KB015].


#### 5. “What should I consider for error handling?

In [101]:
graph= build_rag_graph()

question=input("Enter your question: ")

result= graph.invoke({"question": question})

print(result.get("final_answer", result["answer"]))

2025-10-10 00:50:12,021 - __main__ - INFO - Retrieving snippets for question: “What should I consider for error handling?
2025-10-10 00:50:13,007 - __main__ - INFO - [Retriever] Retrieved 5 snippets
2025-10-10 00:50:13,008 - __main__ - INFO - [GenerateAnswer] Generating answer for: “What should I consider for error handling?
2025-10-10 00:50:13,009 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-10-10 00:50:14,825 - httpx - INFO - HTTP Request: POST https://aiplatform.googleapis.com/v1beta1/projects/bdc-trainings/locations/global/publishers/google/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
2025-10-10 00:50:14,826 - __main__ - INFO - [GenerateAnswer] Generated answer: When addressing error handling, it's important to follow well-defined patterns [KB009, KB029, KB019]...
2025-10-10 00:50:14,827 - __main__ - INFO - [Critique] Critiquing answer...
2025-10-10 00:50:14,828 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
20

When addressing error handling, it's important to follow well-defined patterns [KB009, KB029, KB019].
